<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Managing Stage files with Fusion SQL</h1>
    </div>
</div>

Fusion SQL can be used to manage your workspace groups and workspaces, but it
can also be used to upload, download, and manage files in your workspace group
Stage. We'll show you how to work with files in Stage in this notebook.

## Displaying the Stage Fusion SQL commands

The `SHOW FUSION COMMANDS` displays the commands that are handled by the Fusion
engine. You can use the `LIKE` to filter the commands.

In [1]:
commands = %sql SHOW FUSION COMMANDS LIKE '%stage%'
for cmd in commands:
    print(*cmd, '\n')

## Creating a workspace group

We'll start by creating a workspace group. We can get a region in the US by using the `SHOW REGIONS`
command and the `random` package.

In [2]:
import random

us_regions = %sql SHOW REGIONS LIKE '%us%'

region_id = random.choice(us_regions).ID
region_id

In [3]:
wg_name = 'Fusion Notebook'

In [4]:
%%sql
CREATE WORKSPACE GROUP '{{ wg_name }}' IN REGION ID {{ region_id }}

## Uploading and downloading Stage files

Uploading and downloading files to your Stage is easy with Fusion SQL. The commands are shown below.
```
DOWNLOAD STAGE FILE '<stage-path>' [ IN GROUP { ID '<group-id>' | '<group-name>' } ] [ TO '<local-path>' ]
    [ OVERWRITE ] [ ENCODING '<encoding>' ];

UPLOAD FILE TO STAGE '<stage-path>' [ IN GROUP { ID '<group-id>' | '<group-name>' } ] FROM '<local-path>' [ OVERWRITE ];
```

First we'll create a data file locally that we can work with.

In [5]:
%%writefile mydata.csv
name,age,height
Sue,27,65
Joe,32,70
Max,44,69
Ann,33,64

We can now upload our data file to our workspace group Stage.

In [6]:
%%sql
UPLOAD FILE TO STAGE 'stats.csv' IN GROUP '{{ wg_name }}' FROM 'mydata.csv'

We can list the files in a Stage with the `SHOW STAGE FILES` command.

In [7]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}'

Downloading the file is just as easy as uploading.

In [8]:
%%sql
DOWNLOAD STAGE FILE 'stats.csv' IN GROUP '{{ wg_name }}' TO 'stats.csv'

In [9]:
!cat stats.csv

If you just want to display the contents of the Stage file without saving it to a local
file, you simply leave the `TO` option off the `DOWNLOAD STAGE FILE`.

In [10]:
%%sql
DOWNLOAD STAGE FILE 'stats.csv' IN GROUP '{{ wg_name }}' ENCODING 'utf-8'

## Creating folders

Up to this point we have just worked with files at the root of our Stage. We can use Fusion SQL
to create folder structures as well. This is done with the `CREATE STAGE FOLDER` command.
```
CREATE STAGE FOLDER '<stage-path>' [ IN GROUP { ID '<group-id>' | '<group-name>' } ] [ OVERWRITE ];
```

The following code will create this folder structure:
```
project-1/
project-1/data/
project-2/
project-2/data/
```

In [11]:
for name in ['project-1', 'project-1/data', 'project-2', 'project-2/data']:
    %sql CREATE STAGE FOLDER '{{ name }}' IN GROUP '{{ wg_name }}';

In [12]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}' RECURSIVE

Now that we have a folder structure we can put files into those folders.

In [13]:
%%sql
UPLOAD FILE TO STAGE 'project-1/data/stats.csv' IN GROUP '{{ wg_name }}' FROM 'mydata.csv';
UPLOAD FILE TO STAGE 'project-2/data/stats.csv' IN GROUP '{{ wg_name }}' FROM 'mydata.csv';

Now when we do a recursive listing of our Stage, we'll see the newly created files.

In [14]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}' RECURSIVE

We can list the files at a specific path as well.

In [15]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}' AT 'project-2/data'

## Deleting Stage files and folders

Files and folders can be deleted from a workspace Stage as well.
This is done with the `DROP STAGE FILE` and `DROP STAGE FOLDER` commands.
```
DROP STAGE FILE '<stage-path>' [ IN GROUP { ID '<group-id>' | '<group-name>' } ];

DROP STAGE FOLDER '<stage-path>' [ IN GROUP { ID '<group-id>' | '<group-name>' } ] [ RECURSIVE ];
```

Let's delete the `stats.csv` file at the root of our Stage.

In [16]:
%%sql
DROP STAGE FILE 'stats.csv' IN GROUP '{{ wg_name }}'

In [17]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}'

Now let's delete the `project-2` folder including all of the files in it.

In [18]:
%%sql
DROP STAGE FOLDER 'project-2' IN GROUP '{{ wg_name }}' RECURSIVE

In [19]:
%%sql
SHOW STAGE FILES IN GROUP '{{ wg_name }}' RECURSIVE

## Conclusion

We have demonstrated how to create and delete files and folders in a workspace group Stage
using Fusion SQL. It is also possible to work with Stage files using the SingleStoreDB
Python SDK, see the [API documentation](https://singlestoredb-python.labs.singlestore.com/api.html#stage)
for more details.

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>